In [1]:
import pandas as pd
import numpy as np
from post_processing import error_estimation, merge_estimates_true, compute_val_metrics, under_over_analysis, under_over_by_method, val_metrics_per_iteration, boxplots, boxplots_per_iteration

In [2]:
data = {"method": ["CV", "CV", "Preq", "OOS", "Preq", "CV", "OOS", "OOS", "Preq", "CV"],    
        "model": ["ARIMA", "LSTM", "ARIMA", "ARIMA", "LSTM", "DT", "LSTM", "DT", "DT", "ARIMA"],
        "series": [1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
        "mse": [i**2 for i in range(1, 11)],
        "mae": [i*2 for i in range(1, 11)],
        "weights": [i for i in range(1, 11)]};

data2 = {"model": ["ARIMA", "LSTM", "DT", "ARIMA"],
         "series": [1, 1, 1, 2],
         "mse": [10, 10, 10, 10],
         "mae": [5, 5, 5, 5]};

In [3]:
df = pd.DataFrame(data);
test_df = pd.DataFrame(data2);

In [4]:
new_df = error_estimation(df);

In [5]:
test_df

,model,series,mse,mae
0,ARIMA,1,10,5
1,LSTM,1,10,5
2,DT,1,10,5
3,ARIMA,2,10,5


In [6]:
aggregate_data = pd.merge(left=new_df, right=test_df, on=["model", "series"], suffixes=["_estimate", "_true"])

In [7]:
aggregate_data["model"].unique()

array(['ARIMA', 'DT', 'LSTM'], dtype=object)

In [8]:
aggregate_data

,series,method,model,mse_estimate,mae_estimate,mse_true,mae_true
0,1,CV,ARIMA,1.0,2.0,10,5
1,1,CV,DT,36.0,12.0,10,5
2,1,CV,LSTM,4.0,4.0,10,5
3,1,OOS,ARIMA,16.0,8.0,10,5
4,1,OOS,DT,64.0,16.0,10,5
5,1,OOS,LSTM,49.0,14.0,10,5
6,1,Preq,ARIMA,9.0,6.0,10,5
7,1,Preq,DT,81.0,18.0,10,5
8,1,Preq,LSTM,25.0,10.0,10,5
9,2,CV,ARIMA,100.0,20.0,10,5


In [9]:
l1 = [1, 1, 2];
l2 = [5, 10, 15];

In [10]:
l3 = [L1 * L2 for L2 in l2 for L1 in l1]

In [11]:
l3

[5, 5, 10, 10, 10, 20, 15, 15, 30]

In [12]:
def func(row, arg):

    return row["mse_true"] + row["mae_true"], arg;

In [13]:
aggregate_data.apply(func, args=[1], axis=1)

0    (15, 1)
1    (15, 1)
2    (15, 1)
3    (15, 1)
4    (15, 1)
5    (15, 1)
6    (15, 1)
7    (15, 1)
8    (15, 1)
9    (15, 1)
dtype: object

In [14]:
aggregate_df = merge_estimates_true(new_df, test_df);

In [15]:
aggregate_df

,series,method,model,mse_estimate,mae_estimate,mse_true,mae_true
0,1,CV,ARIMA,1.0,2.0,10,5
1,1,CV,DT,36.0,12.0,10,5
2,1,CV,LSTM,4.0,4.0,10,5
3,1,OOS,ARIMA,16.0,8.0,10,5
4,1,OOS,DT,64.0,16.0,10,5
5,1,OOS,LSTM,49.0,14.0,10,5
6,1,Preq,ARIMA,9.0,6.0,10,5
7,1,Preq,DT,81.0,18.0,10,5
8,1,Preq,LSTM,25.0,10.0,10,5
9,2,CV,ARIMA,100.0,20.0,10,5


In [16]:
res_mse = compute_val_metrics(aggregate_df);

In [17]:
res_mse

,Mean,Median,1st_Quartile,3rd_Quartile,Minimum,Maximum,Standard_deviation,method,model
0,40.50,40.50,15.750,65.250,-9.0,90.0,49.50,CV,ARIMA
1,49.50,49.50,29.250,69.750,9.0,90.0,40.50,CV,ARIMA
2,4.05,4.05,1.575,6.525,-0.9,9.0,4.95,CV,ARIMA
3,4.95,4.95,2.925,6.975,0.9,9.0,4.05,CV,ARIMA
4,26.00,26.00,26.000,26.000,26.0,26.0,0.00,CV,DT
5,26.00,26.00,26.000,26.000,26.0,26.0,0.00,CV,DT
6,2.60,2.60,2.600,2.600,2.6,2.6,0.00,CV,DT
7,2.60,2.60,2.600,2.600,2.6,2.6,0.00,CV,DT
8,-6.00,-6.00,-6.000,-6.000,-6.0,-6.0,0.00,CV,LSTM
9,6.00,6.00,6.000,6.000,6.0,6.0,0.00,CV,LSTM
